In [ ]:
%matplotlib widget

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets
import json
import numpy as np
import h5py
import multiprocessing
import tempfile

from IPython.display import Video

from ophys_etl.types import ExtractROI
from ophys_etl.modules.segmentation.qc_utils.roi_utils import convert_roi_keys
from ophys_etl.modules.segmentation.qc_utils.video_generator import VideoGenerator
from ophys_etl.modules.segmentation.qc_utils.video_display_generator import VideoDisplayGenerator
from ophys_etl.modules.segmentation.processing_log import SegmentationProcessingLog

from evaldb.reader import EvalDBReader
import support_inspection_nb as support

### **Sqlite DB interface for creating notebook inspection manifest**

In [ ]:
sqlite_path = Path("/allen/aibs/informatics/danielk/segmentation_tracking.db")
dbreader = EvalDBReader(sqlite_path)

In [ ]:
all_metadata = dbreader.get_all_metadata()
all_metadata

In [ ]:
ophys_experiment_id = 788422825
inspection_manifest = dbreader.get_inspection_manifest(ophys_experiment_id)
pd.DataFrame.from_records([inspection_manifest["metadata"]])

### **inspection manifest specification**

the above sqlite interface is designed to feed this inspection notebook with an inspection manifest.

But, if one wants to manually specify inspection data sources, the format is:

```
inspection_manifest = {
    "metadata": dictionary (not required) 
    "videos": list of strings that are paths to videos
    "backgrounds": list of strings that are paths to png or pkl background images/graphs
    "processing_logs": list of strings that are paths to hdf5 processing_logs
```
an empty one:
```
inspection_manifest = {
    "videos": [],
    "backgrounds": [],
    "processing_logs": []}
```

### **ROI Viewer**

In [ ]:
widgets.interact(support.roi_viewer, inspection_manifest=widgets.fixed(inspection_manifest), nrows=[1, 2, 3], ncols=[1, 2, 3]);

### **Trace Viewer**

In [ ]:
rois_dict, trace_widgets, roi_drops, movie_widget_list, trace_grouping = support.get_trace_selection_widgets(inspection_manifest)
display(trace_widgets)
b = widgets.interact_manual(support.trace_plot_callback,
                            rois_dict=widgets.fixed(rois_dict),
                            roi_drops=widgets.fixed(roi_drops),
                            movie_widget_list=widgets.fixed(movie_widget_list),
                            trace_grouping=widgets.fixed(trace_grouping),
                            description="plot traces")
b.widget.children[0].description = "plot traces"

### **Video viewing**

In [ ]:
movie_list = inspection_manifest["videos"]

In [ ]:
display_generator = VideoDisplayGenerator()

In [ ]:
%%time
video_generator = VideoGenerator(movie_list[0])

Display full field of view

In [ ]:
%%time
full_fov = video_generator.get_thumbnail_video(origin=(0,0), frame_shape=None, quality=4)

In [ ]:
%%time
Video(**display_generator.display_video(full_fov, width=512, height=512))

Get and display a random thumbnail by hand

In [ ]:
%%time
by_hand_thumbnail = video_generator.get_thumbnail_video(origin=(100, 200), frame_shape=(64, 64),
                                                        quality=5)

In [ ]:
%%time
Video(**display_generator.display_video(by_hand_thumbnail))

Get and display a thumbnail containing an ROI

In [ ]:
keys = list(rois_dict.keys())
raw_roi = rois_dict[keys[0]][6]

In [ ]:
convert_roi_keys([raw_roi])[0].keys()

In [ ]:
keys = list(rois_dict.keys())
raw_roi = convert_roi_keys([rois_dict[keys[0]][6]])[0]
roi = ExtractROI(x=raw_roi['x'], y=raw_roi['y'], width=raw_roi['width'], height=raw_roi['height'],
                 mask=raw_roi['mask_matrix'])

In [ ]:
%%time
roi_thumbnail = video_generator.get_thumbnail_video_from_roi(roi, roi_color=(255,0,0), quality=9)

In [ ]:
%%time
Video(**display_generator.display_video(roi_thumbnail, width=512, height=512))

Use padding kwarg to increase number of pixels on either side of the ROI

In [ ]:
%%time
padded_roi_thumbnail = video_generator.get_thumbnail_video_from_roi(roi, padding=20, roi_color=(255,0,0), quality=7)

In [ ]:
%%time
Video(**display_generator.display_video(padded_roi_thumbnail, width=512, height=512))

Display without the ROI's border

In [ ]:
%%time
no_border_thumbnail = video_generator.get_thumbnail_video_from_roi(roi, quality=9)

In [ ]:
%%time
Video(**display_generator.display_video(no_border_thumbnail, width=512, height=512))

Focus on timesteps where we know (from above) there is activity

In [ ]:
%%time
t0 = 4*60+26
t1 = 4*60+38
timesteps = np.arange(t0*31,t1*31)
active_thumbnail = video_generator.get_thumbnail_video_from_roi(roi, roi_color=(255, 0, 0),
                                                                quality=9, timesteps=timesteps)

In [ ]:
%%time
Video(**display_generator.display_video(active_thumbnail, width=512, height=512))